In [1]:
from collections import namedtuple

In [2]:
Region = namedtuple('Region', ['chrom', 'start', 'end'])
regions = []
with open('../data/VCRome_2.1_design_files/VCRome_2_1_hg38_primary_targets_liftover.bed', 'r') as f:
    for line in f:
        pieces = line.strip().split('\t')
        chrom, start_pos, end_pos = pieces[0], int(pieces[1]), int(pieces[2])
        regions.append(Region(chrom, start_pos, end_pos))
regions = sorted(regions, key=lambda x: (x.chrom, x.start))

with open('../data/VCRome_2.1_design_files/VCRome_2_1_hg38_primary_targets_liftover_ordered.bed', 'w+') as f:
    for region in regions:
        f.write('%s\t%d\t%d\n' % region)

In [3]:
w_start = 50
w_end = 1000

# create new regions
new_regions = []
if regions[0].start-w_start > 0:
    new_regions.append(Region(regions[0].chrom, max(0, regions[0].start-w_end), regions[0].start-w_start))

prev_chrom, prev_end_pos = regions[0].chrom, regions[0].end
for region in regions[1:]:
    if prev_chrom == region.chrom:
        if region.start - prev_end_pos <= 2*w_start:
            pass
        elif region.start - prev_end_pos <= 2*w_end:
            new_regions.append(Region(region.chrom, prev_end_pos+w_start, region.start-w_start))
        else:
            new_regions.append(Region(region.chrom, prev_end_pos+w_start, prev_end_pos+w_end))
            new_regions.append(Region(region.chrom, region.start-w_end, region.start-w_start))
    else:
        new_regions.append(Region(prev_chrom, prev_end_pos+w_start, prev_end_pos+w_end))
        if region.start-w_start > 0:
            new_regions.append(Region(region.chrom, max(0, region.start-w_end), region.start-w_start))
    prev_chrom, prev_end_pos = region.chrom, region.end
new_regions.append(Region(region.chrom, prev_end_pos+w_start, prev_end_pos+w_end))

# check regions for consistency
total = 0
for i in range(len(new_regions)):
    #print(regions[i])
    assert new_regions[i].end > new_regions[i].start
    if i != 0 and new_regions[i].chrom == new_regions[i-1].chrom:
        assert new_regions[i].end > new_regions[i-1].start
    total += new_regions[i].end - new_regions[i].start
print(total/1000000)
           
# write to file
with open('../data/VCRome_2.1_design_files/VCRome_2_1_hg38_primary_targets_liftover_%d_%d.bed' % (w_start, w_end), 'w+') as f:
    for region in new_regions:
        f.write('%s\t%d\t%d\n' % region)




239.957361
